In [1]:
// Mes imports de librairies externes

USE {
    repositories {
        //google()
        // à exécuter en dehors du proxy sinon KO
        mavenCentral()
        //maven("https://artifactory.mycloud.intrabpce.fr/artifactory/g-android-maven-proxy/")
        //maven("https://artifactory.mycloud.intrabpce.fr/artifactory/maven-third-party-android-libs/")
    }
    dependencies {
        val ktor_version = "2.2.4"
        implementation("io.ktor:ktor-client-apache-jvm:$ktor_version")
        implementation("io.ktor:ktor-serialization-kotlinx-json:$ktor_version")
    }
}

In [2]:

import io.ktor.client.*
import io.ktor.client.request.*
import io.ktor.client.statement.*
import kotlinx.coroutines.runBlocking

val client = HttpClient()
runBlocking {
    client.get("https://google.com").bodyAsText()
 }

<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="fr"><head><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"><meta content="/images/branding/googleg/1x/googleg_standard_color_128dp.png" itemprop="image"><title>Google</title><script nonce="VCJlZq_LAplzI__5bOG2oA">(function(){window.google={kEI:'4qdCZIPfMZqnkdUP2f-D8AU',kEXPI:'0,1303427,55982,6058,207,2414,2390,2316,383,246,5,1129120,1197728,163,380600,16114,28684,22430,1362,12316,17583,4998,13228,3847,36218,2226,2872,2891,3926,213,4210,4011,29843,30846,15757,3,576,20583,1,1,2,1528,2304,29062,13064,13659,21223,5812,2545,4094,7596,1,11943,30211,2,14022,6248,19491,5679,1020,31123,4568,6255,23421,1252,5835,14968,4332,7484,27082,8155,6682,699,15969,874,19633,7,1922,9779,8213,14695,13246,6305,2007,18191,5797,4,14432,20206,1622,6755,13129,5831,2306,2106,991,3030,6111,2460,765,8284,8229,1152,1091,269,2615,9481,3570,686,5589,415,2172,23,3585,1642,1406,2130,1330,529,1155,13,477,1155,1610,564,3627,2,214

In [12]:
import io.ktor.client.engine.apache.*
import io.ktor.client.*
import io.ktor.client.plugins.*
import io.ktor.client.request.*
import io.ktor.client.statement.*
import io.ktor.http.*
import kotlinx.serialization.SerialName
import kotlinx.serialization.Serializable
import kotlinx.serialization.decodeFromString
import kotlinx.serialization.json.Json

@Serializable
data class SearchProductList(
    @SerialName("products") val products: List<ProductDetail>? = null
)

@Serializable
data class Product(
    @SerialName("code") val productId: String? = null,
    @SerialName("product") val productDetail: ProductDetail? = null,
)

@Serializable
data class ProductDetail(
    @SerialName("code") val code: String = "",
    @SerialName("product_name_fr") val label: String = "",
    @SerialName("ingredients_text") val ingredientsText: String = "",
    @SerialName("ingredients_text_fr") val ingredientsTextFr: String = "",
    @SerialName("generic_name_fr") val genericName: String = "",
    @SerialName("image_url") val imageUrl: String = "",
    @SerialName("nutrition_grade_fr") val nutriScore: String = "",
    @SerialName("ecoscore_grade") val ecoScore: String = "",
    @SerialName("ecoscore_score") val ecoScoreValue: String = ""
)

class NutriScoreApp() {
    val jsonFormatter = Json { ignoreUnknownKeys = true }

    suspend fun getProduct(productId: String): Product {
        val client = getHttpClient()
        val getProductUrl = "https://fr.openfoodfacts.org/api/v0/produit/$productId.json"
        val productText = client.get(getProductUrl + FIELDS).bodyAsText()
        val product: Product = jsonFormatter.decodeFromString(productText)
        client.close()
        return product
    }

    suspend fun searchProducts(search: String): List<ProductDetail> {
        val client = getHttpClient()
        val searchProductUrl = "https://ssl-api.openfoodfacts.org/cgi/search.pl?search_simple=1&amp;json=1&amp;action=process&amp;sort_by=unique_scans_n"
        val extraParams = "&amp;search_terms=${search.encodeURLPath()}&amp;page=1"
        val searchText = client.get(searchProductUrl + FIELDS + extraParams).bodyAsText()
        val searchResult: SearchProductList = jsonFormatter.decodeFromString(searchText)
        client.close()
        return searchResult.products.orEmpty()
    }

    private fun getHttpClient() = HttpClient {
            install(HttpTimeout) {
                requestTimeoutMillis = 30000
            }
        }

    companion object {
        const val FIELDS = "&amp;fields=selected_images,image_url,product_name,brands,quantity,code,nutrition_grade_fr,ecoscore_grade,product_name_fr,nova_groups,allergens_tags,additives_tags,ingredients_from_palm_oil_n,carbon-footprint-from-known-ingredients_product,carbon-footprint-from-meat-or-fish_product,nutriments"
    }
}

In [16]:
import kotlinx.coroutines.runBlocking

// Exemple de produit :
// Gâteaux au chocolat : 3175681134935
runBlocking {
    val nutriScoreApp = NutriScoreApp()
    val product = nutriScoreApp.getProduct("3175681134935")
    HTML(
        "<h2>${product.productDetail?.label}</h2><br>" +
                "<img src='${product.productDetail?.imageUrl}'/>"
    )
}


Sablé saveur Citron Yuzu au Maltitol

In [27]:
import kotlinx.coroutines.runBlocking

// Exemple de recherche :
runBlocking {
    val search = "pate à tartiner noisette"
    val nutriScoreApp = NutriScoreApp()
    val searchResult = nutriScoreApp.searchProducts(search)

    var searchResultToDisplay: String = ""
    searchResult.forEach {
        searchResultToDisplay += "<h2>${it.label}</h2><br>" +
        if (it.imageUrl.isNotEmpty()) "<img src='${it.imageUrl}'/>" else ""
    }
    HTML(searchResultToDisplay)
}

Pâte à tartiner Nutella noisettes et cacao Nutella Nocciolata Pâte à tartiner au cacao et noisettes Pâte à tartiner noisettes et cacao Pâte à tartiner Nutella noisettes et cacao - 825g Nocciolata Pâte à tartiner Nutella noisettes et cacao - 750g Nocciolata Bio Pâte à tartiner Pâte à tartiner chocolat noisette Nutella pâte à tartiner aux noisettes et au cacao 800g Nocciolata sans lait Pâte à tartiner Milka aux noisettes Pâte à tartiner noisette du Lot et Garonne cacao Nutella Ouf! La pâte à tartiner Pâte à tartiner Pâte à tartiner Chocolat Noisette Pâte à tartiner noisettes et cacao Biscuits Nutella B-ready x15 gaufrettes fourrées - 330g Pâte à tartiner Bio Pâte à tartiner cacao noisette Nocciolata Pâte à tartiner bio sans huile de palme